# 使うライブラリ

In [1]:
!pip install git+https://github.com/pfnet-research/xfeat.git

  Cloning https://github.com/pfnet-research/xfeat.git to /tmp/pip-req-build-b5enlt6l
  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-b5enlt6l


#import

In [2]:
import pandas as pd
import os
import plotly.express as px

from sklearn.model_selection import KFold
from functools import partial
import lightgbm as lgb
import optuna
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer
from pandas.plotting import scatter_matrix

# pandasで作成するプロットの種類をplotlyに変更
pd.options.plotting.backend = 'plotly'

In [3]:
cd /content/drive/MyDrive/signate/signate_PM

/content/drive/MyDrive/signate/signate_PM


In [4]:
ls

exp005.ipynb  submit_sample.csv  train_corr_map.png  v1.csv  v3.csv  v5.csv
README.md     test.csv           train.csv           v2.csv  v4.csv


# data取り込み

In [5]:
train   =pd.read_csv('train.csv')
test   =pd.read_csv('test.csv')
sample =pd.read_csv('submit_sample.csv',names=['id','pm25_mid']) 

train["timestamp"] = pd.to_datetime(train[["year", "month", "day"]])
test["timestamp"] = pd.to_datetime(test[["year", "month", "day"]])

# EDA

In [6]:
train

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,timestamp
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,...,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,2019-01-01
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,...,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,2019-01-01
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,...,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,2019-01-01
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,...,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,2019-01-01
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,...,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,United States,Jacksonville,30.33218,-81.65565,12,0.694,...,2.710,6.125,3.757,12,16.774,22.679,26.058,13.252,16.150,2021-12-31
195937,195938,2021,12,31,United States,Las Vegas,36.17497,-115.13722,14,0.528,...,2.974,6.861,8.354,12,10.432,14.741,15.827,7.078,16.895,2021-12-31
195938,195939,2021,12,31,United States,Milwaukee,43.03890,-87.90647,171,1.975,...,1.087,2.578,0.612,26,2.049,3.531,6.686,5.286,86.299,2021-12-31
195939,195940,2021,12,31,Vietnam,Hanoi,21.02450,105.84117,31,2.613,...,3.058,6.005,6.085,51,1.922,7.443,7.716,4.642,36.523,2021-12-31


In [7]:
train.columns

Index(['id', 'year', 'month', 'day', 'Country', 'City', 'lat', 'lon', 'co_cnt',
       'co_min', 'co_mid', 'co_max', 'co_var', 'o3_cnt', 'o3_min', 'o3_mid',
       'o3_max', 'o3_var', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max',
       'so2_var', 'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'no2_var',
       'temperature_cnt', 'temperature_min', 'temperature_mid',
       'temperature_max', 'temperature_var', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'humidity_var', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'pressure_var',
       'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var', 'dew_cnt', 'dew_min',
       'dew_mid', 'dew_max', 'dew_var', 'pm25_mid', 'timestamp'],
      dtype='object')

In [8]:
test

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,timestamp
0,195942,2019,1,1,Australia,Adelaide,-34.92866,138.59863,155,0.091,...,0.098,2.411,10.213,24.608,26,3.057,8.180,15.841,39.705,2019-01-01
1,195943,2019,1,1,Australia,Sydney,-33.86785,151.20732,23,1.037,...,0.101,2.982,8.261,15.957,47,11.090,15.361,19.227,15.857,2019-01-01
2,195944,2019,1,1,Belgium,Brussels,50.85045,4.34878,28,0.096,...,0.218,1.200,5.802,7.584,44,1.055,4.411,6.955,8.415,2019-01-01
3,195945,2019,1,1,Bosnia and Herzegovina,Sarajevo,43.84864,18.35644,122,0.479,...,0.504,1.172,3.861,2.696,16,0.482,0.484,0.932,0.049,2019-01-01
4,195946,2019,1,1,Brazil,São José dos Campos,-23.17944,-45.88694,158,6.198,...,0.409,1.412,3.132,1.754,165,6.253,11.440,12.473,3.546,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53504,249446,2021,12,31,Turkey,Adana,36.98615,35.32531,97,4.674,...,0.201,2.878,7.397,12.130,86,0.451,3.112,4.512,3.938,2021-12-31
53505,249447,2021,12,31,Turkey,Bursa,40.19559,29.06013,123,17.621,...,0.473,1.491,2.974,1.468,115,3.894,6.823,6.861,2.145,2021-12-31
53506,249448,2021,12,31,Turkey,Trabzon,41.00500,39.72694,150,0.128,...,1.001,3.605,5.040,3.965,12,5.269,8.772,12.661,13.201,2021-12-31
53507,249449,2021,12,31,United Kingdom,London,51.50853,-0.12574,21,0.057,...,0.077,0.803,3.884,3.268,34,10.906,10.959,12.744,0.822,2021-12-31


In [9]:
train.dtypes

id                          int64
year                        int64
month                       int64
day                         int64
Country                    object
City                       object
lat                       float64
lon                       float64
co_cnt                      int64
co_min                    float64
co_mid                    float64
co_max                    float64
co_var                    float64
o3_cnt                      int64
o3_min                    float64
o3_mid                    float64
o3_max                    float64
o3_var                    float64
so2_cnt                     int64
so2_min                   float64
so2_mid                   float64
so2_max                   float64
so2_var                   float64
no2_cnt                     int64
no2_min                   float64
no2_mid                   float64
no2_max                   float64
no2_var                   float64
temperature_cnt             int64
temperature_mi

In [10]:
# for i in train.columns:
#   fig = px.histogram(train[i], x=i)
#   fig.show()
#   print(train[i].value_counts())
#   print()

In [11]:
SelectCategorical().fit_transform(train)

,Country,City,timestamp
0,Australia,Brisbane,2019-01-01
1,Australia,Darwin,2019-01-01
2,Australia,Melbourne,2019-01-01
3,Australia,Newcastle,2019-01-01
4,Australia,Perth,2019-01-01
...,...,...,...
195936,United States,Jacksonville,2021-12-31
195937,United States,Las Vegas,2021-12-31
195938,United States,Milwaukee,2021-12-31
195939,Vietnam,Hanoi,2021-12-31


In [12]:
# for i in test.columns:
#   fig = px.histogram(test[i], x=i)
#   fig.show()
#   print(test[i].value_counts())
#   print()

In [13]:
len(train['City'].unique())

239

In [14]:
len(test['City'].unique())

63

In [15]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
train.groupby('Country').mean()

,id,year,month,day,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,96079.069484,2019.942008,6.618105,15.512907,-29.090034,140.580272,66.077440,1.350904,2.766100,5.984908,33.584097,115.950849,4.383639,15.453733,29.740931,203.471934,58.815771,1.038735,1.972156,6.371348,17.229447,125.017680,1.749241,5.901987,17.615357,72.582385,151.331860,12.423518,17.945728,24.449233,37.039806,157.794908,44.267177,68.636404,90.383619,563.794477,163.939003,985.776507,1009.601801,1030.131699,154.324821,145.308699,0.527510,2.676470,7.427943,16.042676,75.837518,9.332732,13.030085,16.458972,11.466782,37.362265
Belgium,90391.768121,2019.875225,6.377209,15.439596,50.714452,4.816145,65.842409,0.892181,1.991173,4.500611,5.557647,120.550307,5.697792,18.131049,34.670356,230.300921,57.884602,0.947712,1.894285,5.567480,5.585258,139.700325,2.636780,8.414048,22.927064,104.468462,170.126938,9.276668,14.740117,20.365461,29.684738,171.053011,49.415566,71.941718,90.097644,442.394204,178.898666,993.014348,1012.730281,1027.541492,74.587366,161.208078,0.762890,2.779657,6.484547,8.461807,95.941940,6.516061,9.844109,13.140351,8.525323,43.043318
Bosnia and Herzegovina,119584.320304,2020.250814,7.205212,15.929425,44.285415,18.093714,103.494028,1.154192,3.319360,17.504235,91.552336,103.390879,4.203768,22.309561,55.006978,882.371003,99.718784,5.309919,22.721793,80.022484,1709.912532,100.429967,1.717016,8.277117,27.140732,190.777969,128.426710,8.216324,14.091831,21.457384,50.126751,124.798046,45.506748,75.050190,95.181625,679.233055,128.259501,955.691185,1001.740562,1040.699743,1253.548329,127.362649,0.537076,1.522358,4.120000,3.907689,115.040174,5.859718,8.909720,11.821587,9.676436,66.000577
Brazil,99334.288127,2019.994536,6.582712,15.519622,-22.036903,-43.688754,175.753602,1.273528,3.434354,10.913405,43.483453,261.253353,4.614651,17.298044,44.484278,537.817669,129.289121,1.398956,3.120540,10.410835,21.294894,248.244908,1.922573,9.219672,30.031909,250.212977,354.940387,14.010084,19.144188,25.415781,32.843010,353.649280,47.722468,74.239082,91.960549,504.174286,362.801292,984.515267,1011.846080,1032.391067,109.746208,322.074019,0.642520,2.799392,7.724365,14.463509,173.502235,11.195359,14.526311,17.724752,8.825741,46.787973
Canada,100064.639810,2020.003791,6.600948,15.706161,49.249660,-123.119340,117.767773,0.591560,1.343031,3.283928,4.348728,165.450237,3.555551,12.890354,26.018893,147.233176,98.728910,0.500164,1.212576,4.241207,3.952258,171.963981,1.532858,6.228281,18.246841,71.556263,195.651185,7.726330,13.923298,20.062968,48.959313,197.161137,50.941209,76.332502,92.753668,454.776094,206.020853,991.147846,1014.672222,1035.339044,134.320391,158.695735,0.402593,2.481228,9.201606,23.879128,70.252133,6.089476,9.551275,12.851455,9.288471,31.420618
Chile,101366.388421,2020.028273,6.563110,15.441266,-30.570087,-70.729537,72.412992,2.697426,4.827227,12.490663,137.156183,96.400539,5.300863,14.968734,29.167812,225.144338,66.640862,1.342148,2.280886,9.682332,90.756684,105.830360,2.852152,7.378812,20.537026,94.297856,108.308314,8.772798,15.273828,23.020344,52.621474,112.542242,37.183068,58.839355,78.640613,479.101027,112.112757,983.663372,1012.977151,1037.182850,222.480595,102.349041,1.008664,3.615321,7.227003,10.644984,68.410636,1.983629,5.968551,9.672639,13.697905,52.213692
China,98509.960733,2019.976430,6.640888,15.653552,31.858151,114.003009,158.861864,2.282934,5.060159,12.083951,38.071015,193.307334,5.986526,23.184719,51.648528,647.950417,144.701509,1.527609,3.721015,14.083889,57.379438,213.875242,3.213564,11.188337

In [16]:
# corr_mat = train.corr(method='pearson')
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize = (200,200))
# sns.heatmap(corr_mat,
#             vmin=-1.0,
#             vmax=1.0,
#             center=0,
#             annot=True, # True:格子の中に値を表示
#             fmt='.1f',
#             xticklabels=corr_mat.columns.values,
#             yticklabels=corr_mat.columns.values
#            )

# plt.savefig('./train_corr_map.png')
# plt.show()

# 各都市の距離差分の算出
1 都市データのユニークデータを抽出する
2 ユニーク同士の距離を算出する
3 データフレームに追加

In [17]:
# city_train=train[~train.duplicated(subset=['lat','lon','City'],keep='last')][['lat','lon','City']]
# city_test=test[~test.duplicated(subset=['lat','lon','City'],keep='last')][['lat','lon','City']]
# city_all=pd.concat([city_train, city_test]).reset_index(drop=True)

In [18]:
# city_all

In [19]:
# from geopy.distance import geodesic

# dp=[]

# for i in range(len(city_all)):
#   tmp=[]
#   for j in range(len(city_all)):
    
#     if i==j:
#       tmp.append(0)
#     elif i > j:
#       tmp.append(dp[j][i])
#     else:
#       tmp.append(geodesic((city_all['lat'][i],city_all['lon'][i]), (city_all['lat'][j],city_all['lon'][j])).km)
#   dp.append(tmp)

In [20]:
# DB=pd.DataFrame(dp,index=city_all['City'],columns=city_all['City'])

In [21]:
# tmp=[]
# for i in range(len(train)):
#   tmp.append(DB[train['City'][i]].tolist())
# train_tmp=pd.DataFrame(tmp,columns=city_all['City'])
# train=pd.concat([train,train_tmp],axis=1)
# train

In [22]:
# tmp=[]
# for i in range(len(test)):
#   tmp.append(DB[test['City'][i]].tolist())
# test_tmp=pd.DataFrame(tmp,columns=city_all['City'])
# test =pd.concat([test,test_tmp],axis=1)
# test

In [23]:
# fold = KFold(n_splits=30, shuffle=False)
# encoder = TargetEncoder(
#     input_cols=["timestamp"], 
#     target_col="pm25_mid",
#     fold=fold,
#     output_suffix="_re"
#     )

# train = encoder.fit_transform(train)
# test = encoder.fit_transform(test)
# train[["pm25_mid", "timestamp", "timestamp_re"]].head(12)

In [24]:
# print(train['timestamp_re'].value_counts())
# print()
# len(train['timestamp'].unique())

# 時系列groupby

In [25]:
group = 'City'
feature_cols = [ 'co_cnt',
       'co_min', 'co_mid', 'co_max', 'co_var', 'o3_cnt', 'o3_min', 'o3_mid',
       'o3_max', 'o3_var', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max',
       'so2_var', 'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'no2_var',
       'temperature_cnt', 'temperature_min', 'temperature_mid',
       'temperature_max', 'temperature_var', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'humidity_var', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'pressure_var',
       'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var', 'dew_cnt', 'dew_min',
       'dew_mid', 'dew_max', 'dew_var',]

lag = 1
train = pd.concat([
    train, train.groupby(group)[feature_cols].shift(lag).add_prefix(f'shift{lag}_')
], axis=1)

test = pd.concat([
    test, test.groupby(group)[feature_cols].shift(lag).add_prefix(f'shift{lag}_')
], axis=1)

# one-hot-encoding

In [26]:
# カラム指定したい場合は、pd.get_dummies(df_dummy, columns=['workclass'])
df = pd.get_dummies(train,columns=['Country','City'])
test = pd.get_dummies(test,columns=['Country','City'])
df

,id,year,month,day,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,timestamp,shift1_co_cnt,shift1_co_min,shift1_co_mid,shift1_co_max,shift1_co_var,shift1_o3_cnt,shift1_o3_min,shift1_o3_mid,shift1_o3_max,shift1_o3_var,shift1_so2_cnt,shift1_so2_min,shift1_so2_mid,shift1_so2_max,shift1_so2_var,shift1_no2_cnt,shift1_no2_min,shift1_no2_mid,shift1_no2_max,shift1_no2_var,shift1_temperature_cnt,shift1_temperature_min,shift1_temperature_mid,shift1_temperature_max,shift1_temperature_var,shift1_humidity_cnt,shift1_humidity_min,shift1_humidity_mid,shift1_humidity_max,shift1_humidity_var,shift1_pressure_cnt,shift1_pressure_min,shift1_pressure_mid,shift1_pressure_max,shift1_pressure_var,shift1_ws_cnt,shift1_ws_min,shift1_ws_mid,shift1_ws_max,shift1_ws_var,shift1_dew_cnt,shift1_dew_min,shift1_dew_mid,shift1_dew_max,shift1_dew_var,Country_Australia,Country_Belgium,Country_Bosnia and Herzegovina,Country_Brazil,Country_Canada,Country_Chile,Country_China,Country_Croatia,Country_France,Country_Germany,Country_Hungary,Country_India,Country_Iran,Country_Israel,Country_Italy,Country_Japan,Country_Mexico,Country_Netherlands,Country_Poland,Country_Russia,Country_Serbia,Country_South Africa,Country_South Korea,Country_Spain,Country_Taiwan,Country_Thailand,Country_Turkey,Country_United Kingdom,Country_United States,Country_Vietnam,City_Adapazarı,City_Akita,City_Albuquerque,City_Andong,City_Antakya,City_Antwerpen,City_Atlanta,City_Balıkesir,City_Bandar Abbas,City_Bangkok,City_Barcelona,City_Beijing,City_Belfast,City_Bengaluru,City_Berlin,City_Bhopal,City_Bilbao,City_Boise,City_Brescia,City_Brisbane,City_Budapest,City_Busan,City_Bydgoszcz,City_Calama,City_Cardiff,City_Chandigarh,City_Changsha,City_Changwon,City_Changzhou,City_Charleroi,City_Chengdu,City_Chennai,City_Cheongju-si,City_Chiang Mai,City_Chiba,City_Chihuahua,City_Chon Buri,City_Chuncheon,City_Daejeon,City_Darwin,City_Delhi,City_Denver,City_Detroit,City_East London,City_Edinburgh,City_El Paso,City_Erzurum,City_Eskişehir,City_Foshan,City_Fukuoka,City_Fushun,City_Fuzhou,City_Gandhinagar,City_Gasteiz / Vitoria,City_Gdańsk,City_Gent,City_Gifu-shi,City_Granada,City_Guadalajara,City_Guangzhou,City_Guiyang,City_Gwangju,City_Győr,City_Haarlem,City_Haifa,City_Haikou,City_Hamburg,City_Hangzhou,City_Hanoi,City_Harbin,City_Hefei,City_Hegang,City_Hiroshima,City_Hsinchu,City_Huelva,City_Huế,City_Hyderabad,City_Hāpur,City_Hạ Long,City_Incheon,City_Indianapolis,City_Istanbul,City_Jackson,City_Jacksonville,City_Jaipur,City_Jeju City,City_Jerusalem,City_Jieyang,City_Jinan,City_Kagoshima,City_Kanazawa,City_Kaohsiung,City_Kayseri,City_Keelung,City_Khorramabad,City_Kielce,City_Kobe,City_Kochi,City_Kolkata,City_Konya,City_Kraków,City_Kunming,City_Kyoto,City_Kütahya,City_Lampang,City_Las Palmas de Gran Canaria,City_Las Vegas,City_Leeds,City_Lhasa,City_Livorno,City_Liège,City_Los Angeles,City_Los Ángeles,City_Lucknow,City_Madrid,City_Mainz,City_Marseille,City_Melbourne,City_Middelburg,City_Milan,City_Milwaukee,City_Miskolc,City_Miyazaki,City_Mokpo,City_Monterrey,City_Morelia,City_Mumbai,City_Muzaffarnagar,City_Nagano,City_Nagasaki,City_Naha,City_Nakhon Pathom,City_Nanchang,City_Nancy,City_Nanjing,City_Nanning,City_Nantes,City_Naples,City_Nara-shi,City_Netanya,City_Newcastle,City_Ningbo,City_Nizhniy Novgorod,City_Niš,City_Novi Sad,City_Novosibirsk,City_Oaxaca,City_Okayama,City_Oklahoma City,City_Oviedo,City_Pachuca de Soto,City_Palma,City_Patna,City_Perth,City_Phoenix,City_Pohang,City_Poznań,City_Pretoria,City_Puebla,City_Pécs,City_Qingdao,City_Qinhuangdao,City_Qiqihar,City_Qom,City_Quilpué,City_Raleigh,City_Rayong,City_Rom

In [27]:
def feature_engineering(df):
    cols = df.columns.tolist()

    encoder = Pipeline([
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="+",
                               r=2,
                               output_suffix="_plus"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="*",
                               r=2,
                               output_suffix="_mul"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="-",
                               r=2,
                               output_suffix="_minus"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="+",
                               r=3,
                               output_suffix="_plus"),
    ])
    return encoder.fit_transform(df)

In [28]:
# print("After adding interaction features:")
# df_num = feature_engineering(df_num)

In [29]:
df_num=SelectNumerical().fit_transform(df)
test_num=SelectNumerical().fit_transform(test)

#再度EDA

In [37]:
df_num

,id,year,month,day,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,shift1_co_cnt,shift1_co_min,shift1_co_mid,shift1_co_max,shift1_co_var,shift1_o3_cnt,shift1_o3_min,shift1_o3_mid,shift1_o3_max,shift1_o3_var,shift1_so2_cnt,shift1_so2_min,shift1_so2_mid,shift1_so2_max,shift1_so2_var,shift1_no2_cnt,shift1_no2_min,shift1_no2_mid,shift1_no2_max,shift1_no2_var,shift1_temperature_cnt,shift1_temperature_min,shift1_temperature_mid,shift1_temperature_max,shift1_temperature_var,shift1_humidity_cnt,shift1_humidity_min,shift1_humidity_mid,shift1_humidity_max,shift1_humidity_var,shift1_pressure_cnt,shift1_pressure_min,shift1_pressure_mid,shift1_pressure_max,shift1_pressure_var,shift1_ws_cnt,shift1_ws_min,shift1_ws_mid,shift1_ws_max,shift1_ws_var,shift1_dew_cnt,shift1_dew_min,shift1_dew_mid,shift1_dew_max,shift1_dew_var,Country_Australia,Country_Belgium,Country_Bosnia and Herzegovina,Country_Brazil,Country_Canada,Country_Chile,Country_China,Country_Croatia,Country_France,Country_Germany,Country_Hungary,Country_India,Country_Iran,Country_Israel,Country_Italy,Country_Japan,Country_Mexico,Country_Netherlands,Country_Poland,Country_Russia,Country_Serbia,Country_South Africa,Country_South Korea,Country_Spain,Country_Taiwan,Country_Thailand,Country_Turkey,Country_United Kingdom,Country_United States,Country_Vietnam,City_Adapazarı,City_Akita,City_Albuquerque,City_Andong,City_Antakya,City_Antwerpen,City_Atlanta,City_Balıkesir,City_Bandar Abbas,City_Bangkok,City_Barcelona,City_Beijing,City_Belfast,City_Bengaluru,City_Berlin,City_Bhopal,City_Bilbao,City_Boise,City_Brescia,City_Brisbane,City_Budapest,City_Busan,City_Bydgoszcz,City_Calama,City_Cardiff,City_Chandigarh,City_Changsha,City_Changwon,City_Changzhou,City_Charleroi,City_Chengdu,City_Chennai,City_Cheongju-si,City_Chiang Mai,City_Chiba,City_Chihuahua,City_Chon Buri,City_Chuncheon,City_Daejeon,City_Darwin,City_Delhi,City_Denver,City_Detroit,City_East London,City_Edinburgh,City_El Paso,City_Erzurum,City_Eskişehir,City_Foshan,City_Fukuoka,City_Fushun,City_Fuzhou,City_Gandhinagar,City_Gasteiz / Vitoria,City_Gdańsk,City_Gent,City_Gifu-shi,City_Granada,City_Guadalajara,City_Guangzhou,City_Guiyang,City_Gwangju,City_Győr,City_Haarlem,City_Haifa,City_Haikou,City_Hamburg,City_Hangzhou,City_Hanoi,City_Harbin,City_Hefei,City_Hegang,City_Hiroshima,City_Hsinchu,City_Huelva,City_Huế,City_Hyderabad,City_Hāpur,City_Hạ Long,City_Incheon,City_Indianapolis,City_Istanbul,City_Jackson,City_Jacksonville,City_Jaipur,City_Jeju City,City_Jerusalem,City_Jieyang,City_Jinan,City_Kagoshima,City_Kanazawa,City_Kaohsiung,City_Kayseri,City_Keelung,City_Khorramabad,City_Kielce,City_Kobe,City_Kochi,City_Kolkata,City_Konya,City_Kraków,City_Kunming,City_Kyoto,City_Kütahya,City_Lampang,City_Las Palmas de Gran Canaria,City_Las Vegas,City_Leeds,City_Lhasa,City_Livorno,City_Liège,City_Los Angeles,City_Los Ángeles,City_Lucknow,City_Madrid,City_Mainz,City_Marseille,City_Melbourne,City_Middelburg,City_Milan,City_Milwaukee,City_Miskolc,City_Miyazaki,City_Mokpo,City_Monterrey,City_Morelia,City_Mumbai,City_Muzaffarnagar,City_Nagano,City_Nagasaki,City_Naha,City_Nakhon Pathom,City_Nanchang,City_Nancy,City_Nanjing,City_Nanning,City_Nantes,City_Naples,City_Nara-shi,City_Netanya,City_Newcastle,City_Ningbo,City_Nizhniy Novgorod,City_Niš,City_Novi Sad,City_Novosibirsk,City_Oaxaca,City_Okayama,City_Oklahoma City,City_Oviedo,City_Pachuca de Soto,City_Palma,City_Patna,City_Perth,City_Phoenix,City_Pohang,City_Poznań,City_Pretoria,City_Puebla,City_Pécs,City_Qingdao,City_Qinhuangdao,City_Qiqihar,City_Qom,City_Quilpué,City_Raleigh,City_Rayong,City_Rome,City_Sai

# kmeans

In [31]:
# from sklearn.cluster import KMeans
# import numpy as np
# df_array = np.array([df_num['lat'].tolist(),
#                        df_num['lon'].tolist(),
#                        df_num['co_mid'].tolist(),
#                        ], np.int32)
# test_array = np.array([test_num['lat'].tolist(),
#                        test_num['lon'].tolist(),
#                        test_num['co_mid'].tolist(),
#                        ], np.int32)
# df_array = df_array.T
# test_array = test_array.T

# arr=np.concatenate([df_array, test_array])

# pred = KMeans(n_clusters=4).fit_predict(arr)
# df_pred   = pred[:len(df_array)]
# test_pred = pred[len(df_array):]

# df_num['kmeans']=df_pred 
# test_num['kmeans']=test_pred

# # カラム指定したい場合は、pd.get_dummies(df_dummy, columns=['workclass'])
# df_num = pd.get_dummies(df_num,columns=['kmeans'])
# test_num = pd.get_dummies(test_num,columns=['kmeans'])

# パラメータ探索

In [32]:
# LGBM_PARAMS = {
#         "objective": "regression",
#         "metric": "RMSE",
#         "verbosity": -1,
# }


# def objective(df, selector, trial):
#     selector.set_trial(trial)
#     selector.fit(df)
#     input_cols = selector.get_selected_cols()

#     # Hyper Parameter Tuning するパラメータと範囲を設定する
#     lgbm_params = {
#         'num_leaves': trial.suggest_int("num_leaves", 3, 10),
#         'max_depth': trial.suggest_int("max_depth", 3, 10),
#     }
#     lgbm_params.update(LGBM_PARAMS)

#     # Evaluate with selected columns
#     train_set = lgb.Dataset(df[input_cols], label=df["pm25_mid"])
#     scores = lgb.cv(lgbm_params, train_set, num_boost_round=200, stratified=False, seed=1)
    
#     rmse_score = scores['rmse-mean'][-1]
#     return rmse_score


# # 特徴量探索のための説明変数を設定する
# # encoded_train_df は、No.7で作成したDataFrameと同じ
# input_cols = list(df_num.columns)
# input_cols.remove("pm25_mid")


# # 特徴量探索器を作成する
# selector = GBDTFeatureExplorer(
#     input_cols=input_cols,
#     target_col="pm25_mid",
#     fit_once=True,
#     threshold_range=(0.8, 1.0),
#     lgbm_params=LGBM_PARAMS,
# )

# # Hyper Parameter Tuning を行う
# study = optuna.create_study(direction="minimize")
# study.optimize(partial(objective, df_num, selector), n_trials=10)

# # 選択された特徴量を確認する
# selector.from_trial(study.best_trial)
# print("Selected columns:", selector.get_selected_cols())

In [33]:
# print(study.best_params)
# print(study.best_value)

# モデル制作

In [39]:
import numpy as np
cols=[]
for i in df_num.columns:
  if i != 'pm25_mid':
    cols.append(i)
cols

['id',
 'year',
 'month',
 'day',
 'lat',
 'lon',
 'co_cnt',
 'co_min',
 'co_mid',
 'co_max',
 'co_var',
 'o3_cnt',
 'o3_min',
 'o3_mid',
 'o3_max',
 'o3_var',
 'so2_cnt',
 'so2_min',
 'so2_mid',
 'so2_max',
 'so2_var',
 'no2_cnt',
 'no2_min',
 'no2_mid',
 'no2_max',
 'no2_var',
 'temperature_cnt',
 'temperature_min',
 'temperature_mid',
 'temperature_max',
 'temperature_var',
 'humidity_cnt',
 'humidity_min',
 'humidity_mid',
 'humidity_max',
 'humidity_var',
 'pressure_cnt',
 'pressure_min',
 'pressure_mid',
 'pressure_max',
 'pressure_var',
 'ws_cnt',
 'ws_min',
 'ws_mid',
 'ws_max',
 'ws_var',
 'dew_cnt',
 'dew_min',
 'dew_mid',
 'dew_max',
 'dew_var',
 'shift1_co_cnt',
 'shift1_co_min',
 'shift1_co_mid',
 'shift1_co_max',
 'shift1_co_var',
 'shift1_o3_cnt',
 'shift1_o3_min',
 'shift1_o3_mid',
 'shift1_o3_max',
 'shift1_o3_var',
 'shift1_so2_cnt',
 'shift1_so2_min',
 'shift1_so2_mid',
 'shift1_so2_max',
 'shift1_so2_var',
 'shift1_no2_cnt',
 'shift1_no2_min',
 'shift1_no2_mid',
 's

In [40]:
target = df_num["pm25_mid"]
# cols=selector.get_selected_cols()

df_train=df_num
df_test=test_num

In [41]:
for i in cols:
  print(i)
  if i not in df_test.columns:
    df_test[i]=0

id
year
month
day
lat
lon
co_cnt
co_min
co_mid
co_max
co_var
o3_cnt
o3_min
o3_mid
o3_max
o3_var
so2_cnt
so2_min
so2_mid
so2_max
so2_var
no2_cnt
no2_min
no2_mid
no2_max
no2_var
temperature_cnt
temperature_min
temperature_mid
temperature_max
temperature_var
humidity_cnt
humidity_min
humidity_mid
humidity_max
humidity_var
pressure_cnt
pressure_min
pressure_mid
pressure_max
pressure_var
ws_cnt
ws_min
ws_mid
ws_max
ws_var
dew_cnt
dew_min
dew_mid
dew_max
dew_var
shift1_co_cnt
shift1_co_min
shift1_co_mid
shift1_co_max
shift1_co_var
shift1_o3_cnt
shift1_o3_min
shift1_o3_mid
shift1_o3_max
shift1_o3_var
shift1_so2_cnt
shift1_so2_min
shift1_so2_mid
shift1_so2_max
shift1_so2_var
shift1_no2_cnt
shift1_no2_min
shift1_no2_mid
shift1_no2_max
shift1_no2_var
shift1_temperature_cnt
shift1_temperature_min
shift1_temperature_mid
shift1_temperature_max
shift1_temperature_var
shift1_humidity_cnt
shift1_humidity_min
shift1_humidity_mid
shift1_humidity_max
shift1_humidity_var
shift1_pressure_cnt
shift1_pressur

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [42]:
# parameters

learning_rate = 0.05
SEED = 42
lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": learning_rate,
    "num_leaves": 8,
    "min_data_in_leaf": 40,
    #"colsample_bytree": 1.0,
    #"feature_fraction": 1.0,
    #"bagging_freq": 0,
    #"bagging_fraction": 1.0,
    "verbosity": -1,
    "seed": SEED,
}

In [43]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

SEED_SKF = SEED
np.random.seed(SEED)
 
FOLD = 5
skf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=SEED_SKF)
kf = KFold(n_splits=FOLD, shuffle=True, random_state=SEED)

oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
df_feature_importance = pd.DataFrame()

features = cols 

N_CLASSES=1

In [44]:
for fold_, (indexes_trn, indexes_val) in enumerate(kf.split(df_train.values, target.values)):
    print(f"------------------------------ fold {fold_} ------------------------------")
    
    df_trn = df_train.loc[indexes_trn].reset_index(drop=True)
    df_val = df_train.loc[indexes_val].reset_index(drop=True)
    target_trn = target.loc[indexes_trn].reset_index(drop=True)
    target_val = target.loc[indexes_val].reset_index(drop=True)
    
    
    lgb_train = lgb.Dataset(
        df_trn.loc[:, features],
        label=target_trn,
        feature_name=features,
    )
    lgb_valid = lgb.Dataset(
        df_val.loc[:, features],
        label=target_val,
        feature_name=features,
    )
    
    lgb_params["learning_rate"] = learning_rate + np.random.random() * 0.001  # おまじない
    num_round = 999999999
    model = lgb.train(
        lgb_params,
        lgb_train, 
        num_round, 
        valid_sets=[lgb_train, lgb_valid], 
        verbose_eval=300,
        early_stopping_rounds=300 if num_round >= 1e8 else None,
        fobj=None,
        #feval=lgb_metric,
    )
    
    # cv
    prediction_round = model.best_iteration+150 if num_round >= 1e8 else num_round  # おまじない
    oof[indexes_val] = model.predict(df_val[features], num_iteration=prediction_round) 
    
    # feature importance
    df_fold_importance = pd.DataFrame()
    df_fold_importance["feature"] = features
    df_fold_importance["importance"] = model.feature_importance()
    df_fold_importance["fold"] = fold_

    # prediction for test data
    predictions += model.predict(df_test[features], num_iteration=prediction_round) / FOLD
    print()

------------------------------ fold 0 ------------------------------
Training until validation scores don't improve for 300 rounds.
[300]	training's rmse: 21.4264	valid_1's rmse: 21.6371
[600]	training's rmse: 20.5366	valid_1's rmse: 20.9267
[900]	training's rmse: 20.0549	valid_1's rmse: 20.6
[1200]	training's rmse: 19.7353	valid_1's rmse: 20.4189
[1500]	training's rmse: 19.4566	valid_1's rmse: 20.2774
[1800]	training's rmse: 19.2025	valid_1's rmse: 20.1688
[2100]	training's rmse: 18.9641	valid_1's rmse: 20.0716
[2400]	training's rmse: 18.7492	valid_1's rmse: 19.9978
[2700]	training's rmse: 18.541	valid_1's rmse: 19.9298
[3000]	training's rmse: 18.3522	valid_1's rmse: 19.8756
[3300]	training's rmse: 18.171	valid_1's rmse: 19.832
[3600]	training's rmse: 18.0005	valid_1's rmse: 19.7882
[3900]	training's rmse: 17.8312	valid_1's rmse: 19.7466
[4200]	training's rmse: 17.6771	valid_1's rmse: 19.7163
[4500]	training's rmse: 17.5238	valid_1's rmse: 19.6889
[4800]	training's rmse: 17.3765	valid

In [45]:
pd.set_option('display.max_rows', 500)
df_fold_importance 

,feature,importance,fold
0,id,2678,4
1,year,2,4
2,month,1328,4
3,day,937,4
4,lat,1289,4
5,lon,1095,4
6,co_cnt,1240,4
7,co_min,2130,4
8,co_mid,3108,4
9,co_max,1646,4


In [46]:
from sklearn.metrics import mean_squared_error

df_test["prediction"] = predictions.tolist()
score=np.sqrt(mean_squared_error(target, oof))
print("CV score")
print(f"  rmse: {score:8.5f}")

CV score
  rmse: 19.33686


In [47]:
sample=sample.set_axis(['id','pm25_mid'],axis=1)

In [48]:
df_submission = sample.copy()
df_submission["pm25_mid"] = df_submission["id"].map(dict(df_test[["id", "prediction"]].values))
assert not df_submission["pm25_mid"].isna().any()
display(df_submission.head(10))

# make submission file
df_submission.to_csv("v6.csv",header=None,index=None)

,id,pm25_mid
0,195942,25.045024
1,195943,48.572543
2,195944,31.376667
3,195945,64.753052
4,195946,142.488674
5,195947,34.363322
6,195948,110.559311
7,195949,94.955808
8,195950,62.776328
9,195951,107.413377
